In [1]:
import pandas as pd
import json
import requests
from pymongo import MongoClient


In [23]:
# les id des stations
first_station_id = 283164601
second_station_id = 283181971

url = f"https://airqino-api.magentalab.it/v3/getStationHourlyAvg/"

# fonction d'extraction des données de l'API pour les stations 
def extract(url, station_id):
    url_station = url + str(station_id) 
    
    #envoie de requet get a l'API et recuperation en json
    response = requests.get(url_station)  
    data = response.json() 
    
    #conversion de la reponse json en data frame
    df = pd.DataFrame(data["data"])  
    return df 

# extraction des données pour la première station et la deuxième station respectivement a la postion de leur id
df_first_station = extract(url, first_station_id)
df_second_station = extract(url, second_station_id)



In [24]:

# Fonction pour transformer les données extraites
def transform(df_station):
    # Sélectionner les colonnes numériques
    column_num = df_station.select_dtypes(include=['float64', 'int64']).columns
    
    # Remplacer les valeurs manquantes par la moyenne de chaque colonne
    df_station = df_station.fillna(value=df_station[column_num].mean())
    
    # Supprimer les doublons
    df_station = df_station.drop_duplicates()
    
    #creation de la colonne days pour stocker le temps en jour de la colonne timestamp
    df_station["timestamp"] = pd.to_datetime(df_station["timestamp"])
    df_station["days"] = pd.to_datetime(df_station['timestamp'].dt.date)
    
    # selection des colonnes pour le calcule de la moyenne journalière de CO et PM2.5
    column_for_avg = df_station[["days", "CO", "PM2.5"]]
    avg_by_day = column_for_avg.groupby("days").agg({"CO": "mean", "PM2.5": "mean"})
    avg_by_day = avg_by_day.reset_index()

    # Renommer la colonne "PM2.5" en "PM2_5" pour la manipulation facile lors de requete sql 
    avg_by_day.rename(columns={"PM2.5": "PM2_5"}, inplace=True)
    return avg_by_day  

# Transformation des données de la première station et de la deuxième station
transform_first_station = transform(df_first_station)
transform_second_station = transform(df_second_station)


In [25]:

# Fonction pour charger les données transformées dans MongoDB
def load(df, db_name, collection_name):
    # Connexion à MongoDB 
    client = MongoClient('mongodb://localhost:27017/')
    db = client[db_name]  # Accéder à la base de données
    collection = db[collection_name]  # Accéder à la collection

    # Convertir le DataFrame en liste de dictionnaires
    data_dict = df.to_dict("records")

    # Insérer les documents dans la collection MongoDB
    collection.insert_many(data_dict)
    print(f"{len(data_dict)} documents insérés dans {collection_name}")  

# Chargement des données de chaque station dans la base de données MongoDB
load(transform_first_station, "air_quality", "station_283164601")
load(transform_second_station, "air_quality", "station_283181971")

360 documents insérés dans station_283164601
347 documents insérés dans station_283181971
